## 1. Importing Required Libraries

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random
import tkinter as tk
from tkinter import Scrollbar, Text, Button, END

-   `nltk`: Natural Language Toolkit (NLTK) for text processing.
-   `WordNetLemmatizer`: Used to reduce words to their base form (e.g., "running" → "run").
-   `pickle`: Used for loading preprocessed data (like words and classes).
-   `numpy`: Provides numerical operations, particularly arrays.
-   `load_model`: Loads the trained chatbot model from Keras.
-   `json`: Reads intent data from a JSON file.
-   `random`: Selects a random response for variability.
-   `tkinter`: Used to create the chatbot's graphical user interface (GUI).
---

## 2. Ensuring Required NLTK Resources are Available

In [2]:
def check_nltk_resources():
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')
    
    try:
        nltk.data.find('corpora/wordnet')
    except LookupError:
        nltk.download('wordnet')

check_nltk_resources()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


-   This function ensures that the necessary NLTK resources (`punkt` for tokenization and `wordnet` for lemmatization) are available.
-   If not found, it downloads them.
---

## 3. Loading Required Resources

In [3]:
lemmatizer = WordNetLemmatizer()
model = load_model('Model/chatbot_model.h5')
intents = json.loads(open('Data/admission_data.json').read())
words = pickle.load(open('Model/words.pkl', 'rb'))
classes = pickle.load(open('Model/classes.pkl', 'rb'))

-   `lemmatizer`: Initializes the lemmatizer to process user input.
-   `model`: Loads the trained chatbot model (`chatbot_model.h5`).
-   `intents`: Loads predefined intents (questions and responses) from `admission_data.json`.
-   `words`: Loads the vocabulary used for training.
-   `classes`: Loads the intent classes for classification.
---

## 4. Preprocessing User Input

In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

-   Tokenizes the input sentence into words.
-   Converts words to lowercase and applies lemmatization.
-   Example:
    -   **Input**: `"I am applying for admission"`
    -   **Tokenized**: `['I', 'am', 'applying', 'for', 'admission']`
    -   **Lemmatized**: `['i', 'am', 'apply', 'for', 'admission']`
---

## 5. Creating Bag of Words Representation

In [5]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

-   Converts the processed sentence into a **Bag of Words (BoW)** representation.
-   Each word in the predefined `words` list is represented as `1` if it appears in the user input, otherwise `0`.
-   Example:
    -   Words list: `['apply', 'admission', 'scholarship', 'course']`
    -   Input: `"I want to apply for admission"`
    -   BoW: `[1, 1, 0, 0]`
---

## 6. Predicting Intent Class

In [6]:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

-   Uses the trained model to predict the intent based on the BoW representation.
-   Filters predictions with a confidence above `0.25`.
-   Returns a sorted list of intents with their probabilities.
-   Example:
    -   **Predicted Output**:
        ```python
        [{"intent": "admission_process", "probability": "0.89"}]
        ```
---

## 7. Fetching a Response

In [7]:
def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            return random.choice(i['responses'])

-   Matches the predicted intent with the predefined responses in `admission_data.json`.
-   Randomly selects one of the available responses.
-   Example:
    -   If `"admission_process"` is the predicted intent, the bot might respond with:
        -   `"You can apply online on our website."`
---

## 8. Processing User Input and Generating a Response

In [8]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    return get_response(ints, intents)

-   Calls `predict_class()` to determine the intent.
-   Calls `get_response()` to return a relevant response.
---

## 9. Creating the Chatbot GUI

In [9]:
class ChatBotGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("College Admission Chatbot")
        self.root.geometry("500x600")
        self.root.resizable(width=False, height=False)
        self.create_widgets()
        # -   Creates a chatbot UI using **Tkinter**.
        # -   Window title: `"College Admission Chatbot"`.
        # -   Fixed size (`500x600` pixels).

    def create_widgets(self):
        # Create Chat window
        self.chat_log = Text(self.root, bd=1, bg="white", height="8", width="50", font="Arial", wrap='word')
        self.chat_log.config(state='disabled')

        # Bind scrollbar to Chat window
        self.scrollbar = Scrollbar(self.root, command=self.chat_log.yview, cursor="heart")
        self.chat_log['yscrollcommand'] = self.scrollbar.set

        # Create the box to enter message
        self.entry_box = Text(self.root, bd=0, bg="white", width="29", height="5", font="Arial", wrap='word')
        self.entry_box.bind("<Return>", self.send)

        # Create Button to send message
        self.send_button = Button(self.root, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,
                                  bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff', command=self.send)

        # Place all components on the screen
        self.scrollbar.place(x=476, y=6, height=486)
        self.chat_log.place(x=6, y=6, height=486, width=470)
        self.entry_box.place(x=6, y=501, height=90, width=360)
        self.send_button.place(x=370, y=501, height=90)
        # -   **Chat log (`Text`)**: Displays conversation history.
        # -   **Scrollbar (`Scrollbar`)**: Allows scrolling in the chat window.
        # -   **Entry box (`Text`)**: Where users type their messages.
        # -   **Send button (`Button`)**: Sends messages when clicked.

    def send(self, event=None):
        msg = self.entry_box.get("1.0", 'end-1c').strip()
        self.entry_box.delete("0.0", END)

        if msg:
            self.chat_log.config(state='normal')
            self.chat_log.insert(END, "You: " + msg + '\n\n')
            self.chat_log.config(foreground="#442265", font=("Verdana", 12))

            res = chatbot_response(msg)
            self.chat_log.insert(END, "Bot: " + res + '\n\n')

            self.chat_log.config(state='disabled')
            self.chat_log.yview(END)
        # -   Retrieves user input.
        # -   Displays the user's message in the chat log.
        # -   Calls `chatbot_response()` to generate a response.
        # -   Displays the bot’s reply.

-   Creates a chatbot UI using **Tkinter**.
-   Window title: `"College Admission Chatbot"`.
-   Fixed size (`500x600` pixels).
-   **Chat log (`Text`)**: Displays conversation history.
-   **Scrollbar (`Scrollbar`)**: Allows scrolling in the chat window.
-   **Entry box (`Text`)**: Where users type their messages.
-   **Send button (`Button`)**: Sends messages when clicked.
-   Retrieves user input.
-   Displays the user's message in the chat log.
-   Calls `chatbot_response()` to generate a response.
-   Displays the bot’s reply.
---

## Running the Application

In [11]:
if __name__ == "__main__":
    root = tk.Tk()
    gui = ChatBotGUI(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


- Initializes and runs the Tkinter GUI.

This script effectively loads a trained chatbot model, processes user queries, predicts the best intent, and generates responses, all within a simple GUI. 🚀